## Import necessary libraries

In [4]:
import torch
from ultralytics import YOLO
import numpy as np
import os
import random
import glob
import shutil
import json
import yaml
from pprint import pprint
from pathlib import Path

## Check CUDA availability

In [5]:
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA Available:", cuda_available)

# If CUDA is available, print details
if cuda_available:
    DEVICE = torch.cuda.current_device()
    device_name = torch.cuda.get_device_name(DEVICE)
    print(f"Device Name: {device_name}")

else:
    print("CUDA is not available. Please check your GPU drivers and CUDA installation.")

CUDA Available: True
Device Name: NVIDIA GeForce RTX 5070 Ti


## Global Configurations

In [17]:
# Set the random seed for reproducibility
RANDOM_SEED = 300188

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

# Dataset directory
DATASET_DIR = "datasets/Vehicle-License-Plate-Detection"
# YAML config for dataset splits and class names
DATA_YAML = os.path.join(DATASET_DIR, "data.yaml")

# Unique project identifier
PROJECT_NAME = "vehicle-license-plate-detection"
# Which version of the dataset to use
DATASET_VERSION = "final"
# Tag for this set of hyperparameters / training settings
EXPERIMENT_NAME = "imgsz1280-500"

RUN_DIR = os.path.join(PROJECT_NAME, DATASET_VERSION, EXPERIMENT_NAME)

# Base folder for saving evaluation outputs
EVALUATION_DIR = os.path.join(RUN_DIR, "evaluation")
# Base folder for saving model architecture & hyperparameters
ARCHITECTURE_DIR = os.path.join(RUN_DIR, "architecture")

# Location of the best-performing weights file of the trained model 
TRAINED_MODEL_WEIGHTS = os.path.join(RUN_DIR, "weights/best.onnx")

## Dataset Splitting

In [12]:
# ─── CONFIG ────────────────────────────────────────────────────────────
TRAIN_IMG_DIR  = os.path.join(DATASET_DIR, "train", "images")
TRAIN_LBL_DIR  = os.path.join(DATASET_DIR, "train", "labels")
VAL_IMG_DIR    = os.path.join(DATASET_DIR, "valid", "images")
VAL_LBL_DIR    = os.path.join(DATASET_DIR, "valid", "labels")
TEST_IMG_DIR   = os.path.join(DATASET_DIR, "test",  "images")
TEST_LBL_DIR   = os.path.join(DATASET_DIR, "test",  "labels")

# Split ratios
TRAIN_RATIO = 7
VAL_RATIO   = 1

# Ensure reproducibility
random.seed(RANDOM_SEED)

# 1️⃣ Ensure split directories exist
for d in (TRAIN_IMG_DIR, TRAIN_LBL_DIR, VAL_IMG_DIR, VAL_LBL_DIR):
    os.makedirs(d, exist_ok=True)

# 2️⃣ Gather current train & valid images
train_imgs_before = glob.glob(os.path.join(TRAIN_IMG_DIR, "*.jpg")) + \
                    glob.glob(os.path.join(TRAIN_IMG_DIR, "*.png"))
val_imgs_before   = glob.glob(os.path.join(VAL_IMG_DIR,   "*.jpg")) + \
                    glob.glob(os.path.join(VAL_IMG_DIR,   "*.png"))
test_imgs_before   = glob.glob(os.path.join(TEST_IMG_DIR,   "*.jpg")) + \
                    glob.glob(os.path.join(TEST_IMG_DIR,   "*.png"))

# 3️⃣ Compute how many should be in valid after split
total_images = len(train_imgs_before) + len(val_imgs_before) + len(test_imgs_before)
# Desired count for validation based on overall ratio
desired_val   = int(total_images * VAL_RATIO / 10)

# 4️⃣ Shuffle and pick from train
all_train_imgs = train_imgs_before.copy()
random.shuffle(all_train_imgs)

# 5️⃣ Determine how many to move into validation
n_val_to_move = max(0, desired_val - len(val_imgs_before))
val_to_move   = all_train_imgs[:n_val_to_move]

# 6️⃣ Move images & corresponding labels
for img_path in val_to_move:
    fname   = os.path.basename(img_path)
    stem    = os.path.splitext(fname)[0]
    lbl_src = os.path.join(TRAIN_LBL_DIR, stem + ".txt")
    # Move image file to validation folder
    shutil.move(img_path, os.path.join(VAL_IMG_DIR, fname))
    # Move label file if it exists
    if os.path.exists(lbl_src):
        shutil.move(lbl_src, os.path.join(VAL_LBL_DIR, stem + ".txt"))

# 7️⃣ Report final counts
final_train_count = len(glob.glob(os.path.join(TRAIN_IMG_DIR, "*.jpg"))) + \
                    len(glob.glob(os.path.join(TRAIN_IMG_DIR, "*.png")))
final_val_count   = len(glob.glob(os.path.join(VAL_IMG_DIR,   "*.jpg"))) + \
                    len(glob.glob(os.path.join(VAL_IMG_DIR,   "*.png")))
final_test_count  = len(glob.glob(os.path.join(TEST_IMG_DIR,  "*.jpg"))) + \
                    len(glob.glob(os.path.join(TEST_IMG_DIR,  "*.png")))

print("Split complete:")
print(f"  train: {final_train_count} images")
print(f"  valid: {final_val_count} images")
print(f"  test : {final_test_count} images")

Split complete:
  train: 1279 images
  valid: 182 images
  test : 365 images


## Ensure full path dataset in data.yaml

In [13]:
BASE_DIR = Path(os.getcwd()) / DATASET_DIR

# 1️⃣ Load existing YAML
with open(DATA_YAML, "r") as f:
    config = yaml.safe_load(f)

print("Original paths:")
pprint({k: config.get(k) for k in ("train", "val", "test")})

# 2️⃣ Update train/val/test entries to absolute POSIX paths with uppercase drive
for split in ("train", "val", "test"):
    orig = config.get(split, "")
    if orig.startswith(".."):
        # Build new path by appending subpath beyond '..'
        rel = Path(orig)
        parts = rel.parts[1:]  # drop leading '..'
        new_path = BASE_DIR.joinpath(*parts)
    else:
        new_path = Path(orig)
    # Convert to forward-slash style
    path_str = new_path.as_posix()
    # Ensure drive letter is uppercase (e.g. 'c:/...' → 'C:/...')
    if len(path_str) >= 2 and path_str[1] == ':' and path_str[0].islower():
        path_str = path_str[0].upper() + path_str[1:]
    config[split] = path_str

print("\nUpdated paths:")
pprint({k: config.get(k) for k in ("train", "val", "test")})

# 3️⃣ Overwrite data.yaml in place
with open(DATA_YAML, "w") as f:
    yaml.dump(config, f, sort_keys=False)

print(f"\nModified YAML saved directly to '{DATA_YAML}'")


Original paths:
{'test': 'C:/Users/herma/dev/IS/yolo/datasets/Vehicle-License-Plate-Detection/test/images',
 'train': 'C:/Users/herma/dev/IS/yolo/datasets/Vehicle-License-Plate-Detection/train/images',
 'val': 'C:/Users/herma/dev/IS/yolo/datasets/Vehicle-License-Plate-Detection/valid/images'}

Updated paths:
{'test': 'C:/Users/herma/dev/IS/yolo/datasets/Vehicle-License-Plate-Detection/test/images',
 'train': 'C:/Users/herma/dev/IS/yolo/datasets/Vehicle-License-Plate-Detection/train/images',
 'val': 'C:/Users/herma/dev/IS/yolo/datasets/Vehicle-License-Plate-Detection/valid/images'}

Modified YAML saved directly to 'datasets/Vehicle-License-Plate-Detection\data.yaml'


## Image Augmentation

In [9]:
AUGMENT=True

HSV_V = 0.2
DEGREES = 15.0
FLIPUD = 0.25
FLIPLR = 0.25
MOSAIC = 1

## Hyperparameter Tuning

In [10]:
NUMBER_OF_EPOCHS = 500
IMAGE_SIZE = 1280
BATCH_SIZE = 16
PATIENCE = 50
NUM_OF_WORKERS = 8

In [16]:
if AUGMENT:
    HYPERPARAMS = {
        "project": PROJECT_NAME,  # Name of the project
        "name": os.path.join(DATASET_VERSION, EXPERIMENT_NAME),  # Name of the training run
        "data": DATA_YAML, # Path to the dataset configuration file
        "epochs": NUMBER_OF_EPOCHS, # Number of epochs to train for
        "imgsz": IMAGE_SIZE, # Image size for training (640x640 pixels)
        "batch": BATCH_SIZE,  # Batch size
        "device": DEVICE,  # Use GPU if available, otherwise set to -1 for CPU,
        "patience": PATIENCE,  # Number of epochs with no improvement after which training will be stopped
        "cache": "disk",  # Cache images for faster training
        "workers": NUM_OF_WORKERS,    # Number of data loading workers

        "hsv_v":    HSV_V,     # brightness jitter ±20%
        "degrees":  DEGREES,   # rotation ±15°
        "flipud":   FLIPUD,    # vertical flip with 25% chance
        "fliplr":   FLIPLR,    # horizontal flip with 25% chance
    }
else:
    HYPERPARAMS = {
        "project": PROJECT_NAME,  # Name of the project
        "name": os.path.join(DATASET_VERSION, EXPERIMENT_NAME),  # Name of the training run
        "data": DATA_YAML, # Path to the dataset configuration file
        "epochs": NUMBER_OF_EPOCHS, # Number of epochs to train for
        "imgsz": IMAGE_SIZE, # Image size for training (640x640 pixels)
        "batch": BATCH_SIZE,  # Batch size
        "device": DEVICE,  # Use GPU if available, otherwise set to -1 for CPU,
        "patience": PATIENCE,  # Number of epochs with no improvement after which training will be stopped
        "cache": "disk",  # Cache images for faster training
        "workers": NUM_OF_WORKERS,    # Number of data loading workers
    }

## Model Training

In [ ]:
if __name__ == "__main__":
    # Initialize YOLOv8n model using the pre-trained weights
    model = YOLO("yolo_pretrained/yolov8m.pt")  # Load a pretrained YOLOv8 model

    # Start training with the pre-trained weights as the initialization
    results = model.train(
        **HYPERPARAMS,  # Unpack hyperparameters
    )

    # Export the trained weights to ONNX format once training completes:
    model.export(format='onnx')

NameError: name 'YOLO' is not defined

## Save Model Architecture & Hyperparamaters used

In [18]:
os.makedirs(ARCHITECTURE_DIR, exist_ok=True)

# 1️⃣ Save hyperparameters as JSON
hyp_path = os.path.join(ARCHITECTURE_DIR, "hyperparameters.json")
with open(hyp_path, "w") as f:
    json.dump(HYPERPARAMS, f, indent=2)
print(f"→ Hyperparameters written to {hyp_path}")

# 2️⃣ Save the model architecture (as text)
arch_path = os.path.join(ARCHITECTURE_DIR, "model_architecture.txt")
with open(arch_path, "w") as f:
    f.write(str(model.model))  
print(f"→ Model architecture written to {arch_path}")

# 3️⃣ (Optional) Copy the best weights over
best_weights = os.path.join(ARCHITECTURE_DIR, "weights", "best.onnx")
if os.path.isfile(best_weights):
    os.replace(best_weights, os.path.join(ARCHITECTURE_DIR, "best_{EXPERIMENT_NAME}.onnx"))
    print("→ Copied best.onnx with custom name")

→ Hyperparameters written to vehicle-license-plate-detection\final\imgsz1280-500-augment-n\architecture\hyperparameters.json
→ Model architecture written to vehicle-license-plate-detection\final\imgsz1280-500-augment-n\architecture\model_architecture.txt


## Testing Dataset Evaluation

In [20]:
TRAINED_MODEL_WEIGHTS = 'vehicle-license-plate-detection/near-complete/imgsz1280-500/weights/best.pt'
IMAGE_SIZE = 1280

In [21]:
if __name__ == "__main__":
    # 1️⃣ Load the model once, with task pre-declared
    model = YOLO(TRAINED_MODEL_WEIGHTS, task="detect")

    # 2️⃣ Evaluate at several confidence thresholds
    for conf in (0.25, 0.50, 0.75):
        model.val(
            data=DATA_YAML,
            split="test",
            project=EVALUATION_DIR,      
            name=f"{conf:.2f}",          
            exist_ok=True,
            workers=NUM_OF_WORKERS,
            conf=conf,                   
            device=DEVICE,
            save_json=True,
            half=False,
            imgsz=IMAGE_SIZE,
        )
        print(f"Finished evaluation at conf={conf:.2f}")

Ultralytics 8.3.146  Python-3.13.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 3297.01561.6 MB/s, size: 884.4 KB)


val: Scanning C:\Users\herma\dev\IS\yolo\datasets\Vehicle-License-Plate-Detection\test\labels.cache... 365 images, 0 backgrounds, 0 corrupt: 100%|██████████| 365/365 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00,  7.76it/s]


                   all        365       1972      0.897      0.818      0.888        0.7
              carplate        363        674      0.941      0.822      0.896      0.637
               vehicle        365       1298      0.854      0.814       0.88      0.763
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving vehicle-license-plate-detection\final\imgsz1280-500\evaluation\0.25\predictions.json...
Results saved to vehicle-license-plate-detection\final\imgsz1280-500\evaluation\0.25
Finished evaluation at conf=0.25
Ultralytics 8.3.146  Python-3.13.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 3730.82375.6 MB/s, size: 431.1 KB)


val: Scanning C:\Users\herma\dev\IS\yolo\datasets\Vehicle-License-Plate-Detection\test\labels.cache... 365 images, 0 backgrounds, 0 corrupt: 100%|██████████| 365/365 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00,  8.00it/s]


                   all        365       1972      0.945      0.772      0.872      0.696
              carplate        363        674      0.961      0.774      0.876       0.63
               vehicle        365       1298      0.928      0.769      0.869      0.763
Speed: 0.4ms preprocess, 2.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Saving vehicle-license-plate-detection\final\imgsz1280-500\evaluation\0.50\predictions.json...
Results saved to vehicle-license-plate-detection\final\imgsz1280-500\evaluation\0.50
Finished evaluation at conf=0.50
Ultralytics 8.3.146  Python-3.13.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 4100.61767.2 MB/s, size: 1201.3 KB)


val: Scanning C:\Users\herma\dev\IS\yolo\datasets\Vehicle-License-Plate-Detection\test\labels.cache... 365 images, 0 backgrounds, 0 corrupt: 100%|██████████| 365/365 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00,  8.14it/s]


                   all        365       1972      0.981      0.674      0.829       0.68
              carplate        363        674      0.989      0.671      0.828      0.609
               vehicle        365       1298      0.973      0.677       0.83       0.75
Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving vehicle-license-plate-detection\final\imgsz1280-500\evaluation\0.75\predictions.json...
Results saved to vehicle-license-plate-detection\final\imgsz1280-500\evaluation\0.75
Finished evaluation at conf=0.75


In [68]:
# Set the random seed for reproducibility
RANDOM_SEED = 300188

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

# Dataset directory
DATASET_DIR = "datasets/Vehicle-License-Plate-Detection"
# YAML config for dataset splits and class names
DATA_YAML = os.path.join(DATASET_DIR, "data.yaml")

# Unique project identifier
PROJECT_NAME = "vehicle-license-plate-detection"
# Which version of the dataset to use
DATASET_VERSION = "baseline"
# Tag for this set of hyperparameters / training settings
EXPERIMENT_NAME = "default"

RUN_DIR = os.path.join(PROJECT_NAME, DATASET_VERSION, EXPERIMENT_NAME)

# Base folder for saving evaluation outputs
EVALUATION_DIR = os.path.join(RUN_DIR, "evaluation")
# Base folder for saving model architecture & hyperparameters
ARCHITECTURE_DIR = os.path.join(RUN_DIR, "architecture")

# Location of the best-performing weights file of the trained model 
TRAINED_MODEL_WEIGHTS = os.path.join(RUN_DIR, "weights/best.onnx")

In [69]:
TRAINED_MODEL_WEIGHTS = 'vehicle-license-plate-detection/baseline/default/weights/best.pt'
IMAGE_SIZE = 640

In [70]:
if __name__ == "__main__":
    # 1️⃣ Load the model once, with task pre-declared
    model = YOLO(TRAINED_MODEL_WEIGHTS, task="detect")

    # 2️⃣ Evaluate at several confidence thresholds
    for conf in (0.25, 0.50, 0.75):
        model.val(
            data=DATA_YAML,
            split="test",
            project=EVALUATION_DIR,      
            name=f"{conf:.2f}",          
            exist_ok=True,
            workers=NUM_OF_WORKERS,
            conf=conf,                   
            device=DEVICE,
            save_json=True,
            half=False,
            imgsz=IMAGE_SIZE,
        )
        print(f"Finished evaluation at conf={conf:.2f}")

Ultralytics 8.3.146  Python-3.13.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 3882.41451.7 MB/s, size: 884.4 KB)


val: Scanning C:\Users\herma\dev\IS\yolo\datasets\Vehicle-License-Plate-Detection\test\labels.cache... 365 images, 0 backgrounds, 0 corrupt: 100%|██████████| 365/365 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00, 11.12it/s]


                   all        365       1972      0.887      0.663      0.801      0.597
              carplate        363        674      0.893      0.622      0.775      0.497
               vehicle        365       1298       0.88      0.704      0.827      0.696
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 1.6ms postprocess per image
Saving vehicle-license-plate-detection\baseline\default\evaluation\0.25\predictions.json...
Results saved to vehicle-license-plate-detection\baseline\default\evaluation\0.25
Finished evaluation at conf=0.25
Ultralytics 8.3.146  Python-3.13.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 3826.52573.2 MB/s, size: 431.1 KB)


val: Scanning C:\Users\herma\dev\IS\yolo\datasets\Vehicle-License-Plate-Detection\test\labels.cache... 365 images, 0 backgrounds, 0 corrupt: 100%|██████████| 365/365 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00,  9.78it/s]


                   all        365       1972      0.934      0.622       0.79      0.599
              carplate        363        674      0.937      0.576      0.762      0.499
               vehicle        365       1298      0.931      0.668      0.818      0.699
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Saving vehicle-license-plate-detection\baseline\default\evaluation\0.50\predictions.json...
Results saved to vehicle-license-plate-detection\baseline\default\evaluation\0.50
Finished evaluation at conf=0.50
Ultralytics 8.3.146  Python-3.13.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 3392.81503.4 MB/s, size: 1201.3 KB)


val: Scanning C:\Users\herma\dev\IS\yolo\datasets\Vehicle-License-Plate-Detection\test\labels.cache... 365 images, 0 backgrounds, 0 corrupt: 100%|██████████| 365/365 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00,  9.70it/s]


                   all        365       1972      0.966      0.555      0.765      0.599
              carplate        363        674       0.96      0.496      0.729      0.497
               vehicle        365       1298      0.973      0.615        0.8      0.701
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 1.3ms postprocess per image
Saving vehicle-license-plate-detection\baseline\default\evaluation\0.75\predictions.json...
Results saved to vehicle-license-plate-detection\baseline\default\evaluation\0.75
Finished evaluation at conf=0.75
